<a href="https://colab.research.google.com/github/sadjadasghari/Colab/blob/main/Diagnostics_Getting_Started_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  
  <td>
    <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/>
  </td>

----

# Model Diagnostics


Throughout the process of training your machine learning (ML) model, you may want to investigate your model's failures in order to understand which areas need improvement. Looking at an error analysis after each training iteration can help you understand whether you need to revise your annotations, make your ontology more clear, or create more training data that targets a specific area.
Labelbox now offers a Model Diagnostics tool that analyzes the performance of your model's predictions in a single interface.
With Model Diagnostics, you can:
*   Inspect model behavior across experiments
*   Adjust model hyperparameters and visualize model failures
*   Use the Python SDK to create the analysis pipeline

## How it works

Configuring Model Diagnostics is all done via the SDK. We have created a Google colab notebook to demonstrate this process. The notebook also includes a section that leverages MAL in order to quickly create ground truth annotations.
An Experiment is a specific instance of a model generating output in the form of predictions.
In Labelbox, the `Model` object represents your ML model and it is what you'll be performing experiments on. It references a set of annotations specified by an ontology. 
The `Model Run` object represents the experiment itself. It is a specific instance of a `Model` with preconfigured hyperparameters (training data). You can upload inferences across each `Model Run`, filter by IoU score, and compare your model's predictions against the annotations from your training data.

## Steps

1. Have a set of ground truth labels in a project
2. Install beta release of the SDK (SDK versions that are compatible with Model Diagnostics will have a "b" in the version name. The first SDK release to support this is 2.5b0.)
3. Create a `Model`
4. Create a `Model Run`
5. Compute predictions
6. Compute model performance metrics
7. Upload labels, predictions, and metrics
8. Navigate to the `Models` tab on Labelbox

## Best practices
Currently there is a limit of 2000 images per model run. We suggest uploading lower performing examples from your test set.

## Environment Setup

Install dependencies

In [ ]:
!pip install labelbox==2.7b1 \
             requests \
             ndjson \
             scikit-image \
             PILLOW \
             tensorflow \
             opencv-python \
             base36

     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 10.1MB 23.8MB/s 


In [ ]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout mea-dev
    !mv labelbox-python/examples/model_assisted_labeling/*.py .
    !mv labelbox-python/examples/model_assisted_labeling/mapillary_sample.csv .

Cloning into 'labelbox-python'...
remote: Enumerating objects: 4812, done.
remote: Counting objects: 100% (1714/1714), done.
remote: Compressing objects: 100% (544/544), done.
remote: Total 4812 (delta 1181), reused 1623 (delta 1148), pack-reused 3098
Receiving objects: 100% (4812/4812), 74.32 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (3224/3224), done.
Branch 'mea-dev' set up to track remote branch 'mea-dev' from 'origin'.
Switched to a new branch 'mea-dev'


Import libraries

In [ ]:
from io import BytesIO
from getpass import getpass
import uuid
import numpy as np
from PIL import Image
import requests
from tqdm import notebook
from collections import defaultdict
import ndjson
import os

from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, MALPredictionImport, DataRow
from labelbox.data.metrics.iou import datarow_miou

try:
    from image_model import predict, load_model, class_mappings
    from ndjson_utils import (
        create_boxes_ndjson, 
        create_polygon_ndjson, 
        create_mask_ndjson, 
        create_point_ndjson
    )
except ModuleNotFoundError: 
    # !git clone https://github.com/Labelbox/labelbox-python.git
    # !cd labelbox-python && git checkout mea-dev
    # !mv labelbox-python/examples/model_assisted_labeling/*.py .
    # !mv labelbox-python/examples/model_assisted_labeling/mapillary_sample.csv .
    raise Exception("You will need to run from the labelbox-python git repo")

Configure client

In [ ]:
API_KEY = None
PROJECT_NAME = "Diagnostics Demo"
MODEL_NAME = "MSCOCO-Mapillary"
MODEL_VERSION = "0.0.0"

In [ ]:
client = Client(api_key=API_KEY)
load_model() # initialize Tensorflow Model

## Setup a project

In [ ]:
class_mappings = {
    1: {"name": 'person', "kind": "bbox"},
    2: {"name": 'bicycle', "kind": "segmentation"},
    3: {"name": 'car', "kind": "bbox"},
    4: {"name": 'motorcycle', "kind": "bbox"},
    6: {"name": 'bus', "kind": "polygon"},
    7: {"name": 'train', "kind": "polygon"},
    8: {"name": 'truck', "kind": "polygon"},
    10: {"name": 'traffic light', "kind": "point"},
    11: {"name": 'fire hydrant', "kind": "bbox"},
    13: {"name": 'stop sign', "kind": "segmentation"},
    14: {"name": 'parking meter', "kind": "point"},
}

In [ ]:
print(f"Setting up: {PROJECT_NAME}")

# --- setup ontology
tools = []
for target in class_mappings.values():
    if target["kind"] == "bbox":
        tool = Tool.Type.BBOX
    elif target["kind"] == "polygon":
        tool = Tool.Type.POLYGON
    elif target["kind"] == "segmentation":
        tool = Tool.Type.SEGMENTATION
    elif target["kind"] == "point":
        tool = Tool.Type.POINT
    else:
        raise ValueError("Type not supported")

    tools.append(Tool(tool=tool, name=target["name"]))

ontology_builder = OntologyBuilder(tools=tools)

# --- setup project
project = client.create_project(name=PROJECT_NAME)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())

# --- setup dataset
# load mapillary sample
with open('mapillary_sample.csv', 'r') as file:
  rows = [row[:-1].split(",") for row in file.readlines()]
  data_rows = [{DataRow.row_data: row[0], DataRow.external_id: row[1]} for row in rows]

dataset = client.create_dataset(name="Mapillary Diagnostics Demo")
task = dataset.create_data_rows(data_rows)
task.wait_till_done()
print(f"Dataset Created: {dataset.uid}")

project.datasets.connect(dataset)
project_id = project.uid

ontology = project.ontology()
schema_lookup = {tool.name: tool for tool in ontology.tools()}

Setting up: Diagnostics Demo


NameError: ignored

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [ ]:
RUN_MAL = True

if RUN_MAL:
    datarows = project.export_queued_data_rows()
    project.enable_model_assisted_labeling()

In [ ]:
predictions = []

for datarow in notebook.tqdm(datarows):
    np_image_bytes = np.array([requests.get(datarow["rowData"]).content])
    datarow_id = datarow["id"]
    w, h = Image.open(BytesIO(np_image_bytes[0])).size

    
    prediction = predict(np_image_bytes, min_score=0.25, height=h, width=w)
    
    boxes, classes, seg_masks = prediction["boxes"], prediction["class_indices"], prediction["seg_masks"]
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        
        if class_idx in class_mappings:
            
            class_name = class_mappings.get(class_idx)
            schema = schema_lookup.get(class_name["name"], None)
            schema_id = schema.feature_schema_id

            if schema.tool == Tool.Type.POLYGON:
                predictions.append(
                    create_polygon_ndjson(datarow_id, schema_id, seg)
                    )
            elif schema.tool == Tool.Type.BBOX:
                predictions.append(
                    create_boxes_ndjson(datarow_id, schema_id, *box)
                    )
            elif schema.tool.name == Tool.Type.POINT:
                predictions.append(
                    create_point_ndjson(datarow_id, schema_id, *box)
                )
            elif schema.tool.name == Tool.Type.SEGMENTATION:
                predictions.append(
                    create_mask_ndjson(client, datarow_id, schema_id, seg, (255, 0, 0))
                )
            else:
                raise ValueError

## **Optional** - Create labels with Model Assisted Labeling

* Pre-label image so that we can quickly create ground truth
* Create ground truth data for Model Diagnostics
* Click on link below to label

In [ ]:
if RUN_MAL:
    upload_task = MALPredictionImport.create_from_objects(client, project.uid, f'mal-import-{uuid.uuid4()}', predictions)
    upload_task.wait_until_done()
    print(upload_task.state , '\n')

In [ ]:
print(f"https://app.labelbox.com/go-label/{project.uid}")

## Export Labels

We do not support `Skipped` labels and have a limit of **2000**

In [ ]:
MAX_LABELS = 2000
labels = [l for l in requests.get(project.export_labels()).json() if l["Label"]][:MAX_LABELS]

## Setup Model & Model Run

In [ ]:
lb_model = client.create_model(name = MODEL_NAME, ontology_id = project.ontology().uid)
lb_model_run = lb_model.create_model_run(MODEL_VERSION)

Select label ids to upload

In [ ]:
lb_model_run.upsert_labels([label['ID'] for label in labels])

### Compute Metrics

In [ ]:
# Note that the `datarow_miou` function downloads segmentation masks
# For large projects this logic should be wrapped in a threadpool for faster execution.

metric_annotations = []
grouped_predictions = defaultdict(list)
labeled_datarows = {label["DataRow ID"] for label in labels}
labeled_predictions = [pred for pred in predictions if pred['dataRow']['id'] in labeled_datarows] # filter to datarows with uploaded labels

for prediction in labeled_predictions:
    grouped_predictions[prediction['dataRow']['id']].append(prediction)
    
for label in labels:
    datarow_id = label['DataRow ID']
    # ignore skipped
    if len(grouped_predictions[datarow_id]):
        score = datarow_miou(label, grouped_predictions[datarow_id])
        if score is None:
            continue
        
        metric_annotations.append(  {
            "uuid" : str(uuid.uuid4()),
            "dataRow" : {
                "id": datarow_id,
            },
            "metricValue" : score
            }
        )

In [ ]:
upload_task = lb_model_run.add_predictions(f'diagnostics-import-{uuid.uuid4()}', labeled_predictions + metric_annotations)
upload_task.wait_until_done()
print(upload_task.state)

### Open Model Run

In [ ]:
for idx, annotation_group in enumerate(lb_model_run.annotation_groups()):
    if idx == 5:
        break
    print(annotation_group.url)